In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_06 = pd.read_csv(folderpath + 'data_op_06.csv')
data_op_06.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
3,1,4,20.0031,0.7005,0.0,1488.44,1249.18,9.35,2323.85,314.84,2388.07,0.02,6,1
5,1,6,20.0032,0.7017,0.0,1480.46,1258.90,9.35,2323.94,315.36,2388.05,0.02,6,1
22,1,23,20.0025,0.7011,0.0,1482.68,1252.00,9.35,2323.91,314.87,2388.10,0.02,6,1
30,1,31,20.0045,0.7006,0.0,1483.93,1256.88,9.35,2323.91,314.94,2388.12,0.02,6,2
44,1,45,20.0062,0.7000,0.0,1477.33,1250.58,9.35,2323.91,315.06,2388.09,0.02,6,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_06 = tf_remove_sensor_15.fit_transform(data_op_06)
data_op_06.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
3,1,4,20.0031,0.7005,0.0,1488.44,1249.18,9.35,2323.85,314.84,2388.07,6,1
5,1,6,20.0032,0.7017,0.0,1480.46,1258.90,9.35,2323.94,315.36,2388.05,6,1
22,1,23,20.0025,0.7011,0.0,1482.68,1252.00,9.35,2323.91,314.87,2388.10,6,1
30,1,31,20.0045,0.7006,0.0,1483.93,1256.88,9.35,2323.91,314.94,2388.12,6,2
44,1,45,20.0062,0.7000,0.0,1477.33,1250.58,9.35,2323.91,315.06,2388.09,6,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_06)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
3,1488.44,1249.18,9.35,2323.85,314.84,2388.07,1
5,1480.46,1258.90,9.35,2323.94,315.36,2388.05,1
22,1482.68,1252.00,9.35,2323.91,314.87,2388.10,1
30,1483.93,1256.88,9.35,2323.91,314.94,2388.12,2
44,1477.33,1250.58,9.35,2323.91,315.06,2388.09,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[ 0.42240019, -0.57734709,  1.        , -1.33580941, -0.01630742,
        -0.65158097],
       [-0.9665274 ,  0.66883265,  1.        , -0.63627355,  1.04756387,
        -0.80400561],
       [-0.58013401, -0.21580111,  1.        , -0.86945217,  0.04506977,
        -0.42294403],
       ...,
       [ 0.90452168,  0.80857915,  1.        , -0.63627355, -0.60962026,
        -0.42294403],
       [ 0.66781221, -0.15810761,  1.        , -0.0921901 , -0.50732494,
         0.03432987],
       [ 1.2474023 ,  1.29064045,  1.        , -0.01446389, -0.54824307,
        -0.49915634]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 1, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))}
                  }

In [ ]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=50, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=493 ..........................................
[CV] hidden_layer_sizes=493 ..........................................
[CV] hidden_layer_sizes=493 ..........................................
[CV] hidden_layer_sizes=493 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_layer_sizes=356 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed: 33.4min


[CV] hidden_layer_sizes=677 ..........................................
[CV] . hidden_layer_sizes=801, score=0.6224188790560472, total=16.4min
[CV] hidden_layer_sizes=677 ..........................................
[CV] . hidden_layer_sizes=482, score=0.6124260355029586, total=33.1min
[CV] hidden_layer_sizes=677 ..........................................
[CV] . hidden_layer_sizes=572, score=0.6528803545051699, total=33.4min
[CV] hidden_layer_sizes=934 ..........................................
[CV] . hidden_layer_sizes=279, score=0.6297935103244838, total=13.7min
[CV] hidden_layer_sizes=934 ..........................................
[CV] . hidden_layer_sizes=768, score=0.6528803545051699, total=34.1min
[CV] hidden_layer_sizes=934 ..........................................
[CV] . hidden_layer_sizes=768, score=0.6094674556213018, total=34.1min
[CV] hidden_layer_sizes=934 ..........................................
[CV] . hidden_layer_sizes=768, score=0.6617429837518464, total=34.2min
[CV] h

[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed: 77.9min


[CV] hidden_layer_sizes=266 ..........................................
[CV] . hidden_layer_sizes=526, score=0.6115214180206795, total=18.5min
[CV] hidden_layer_sizes=150 ..........................................
[CV] .. hidden_layer_sizes=230, score=0.654357459379616, total=14.3min
[CV] hidden_layer_sizes=150 ..........................................
[CV] . hidden_layer_sizes=230, score=0.6410635155096012, total=16.1min
[CV] hidden_layer_sizes=150 ..........................................
[CV] . hidden_layer_sizes=677, score=0.6573116691285081, total=44.6min
[CV] hidden_layer_sizes=150 ..........................................
[CV] . hidden_layer_sizes=553, score=0.6528803545051699, total=25.9min
[CV] hidden_layer_sizes=150 ..........................................
[CV] . hidden_layer_sizes=934, score=0.6094674556213018, total=41.8min
[CV] hidden_layer_sizes=150 ..........................................
[CV] . hidden_layer_sizes=230, score=0.6558345642540621, total=16.8min
[CV] h

[Parallel(n_jobs=100)]: Done 402 out of 500 | elapsed: 104.7min remaining: 25.5min


[CV] . hidden_layer_sizes=305, score=0.6100443131462334, total= 9.8min
[CV] .. hidden_layer_sizes=621, score=0.654357459379616, total=32.3min
[CV] . hidden_layer_sizes=400, score=0.6356932153392331, total=16.2min
[CV] . hidden_layer_sizes=305, score=0.6691285081240768, total=11.6min
[CV] .. hidden_layer_sizes=435, score=0.606508875739645, total=17.2min
[CV] . hidden_layer_sizes=435, score=0.6602658788774003, total=18.9min
[CV] . hidden_layer_sizes=168, score=0.6617429837518464, total=16.1min
[CV] . hidden_layer_sizes=400, score=0.6469719350073855, total=17.7min
[CV] .. hidden_layer_sizes=400, score=0.616519174041298, total=19.0min
[CV] . hidden_layer_sizes=266, score=0.6528803545051699, total=29.4min
[CV] . hidden_layer_sizes=364, score=0.6661742983751846, total=25.6min
[CV] . hidden_layer_sizes=168, score=0.6144756277695717, total=18.0min
[CV] . hidden_layer_sizes=364, score=0.6474926253687315, total=26.1min
[CV] . hidden_layer_sizes=845, score=0.6262924667651403, total=51.1min
[CV] .

## Savel results, models and pipeline to a .pkl file 

In [ ]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [ ]:
experiment.save_models(random_search)
experiment.save_pipeline(data_preprocessing)